# Step 2: Embedding-based Model

This notebook implements an embedding-based model for predicting human preference of LLM responses.
We will use a pre-trained sentence embedding model to construct prompt+response embeddings and train a classifier.

## 1. Install and Import Libraries

In [2]:
!pip install sentence-transformers torch

   ---------------------------------------- 0.0/488.0 kB ? eta -:--:--
   ------------ --------------------------- 153.6/488.0 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 488.0/488.0 kB 6.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
import re

c:\Users\user\anaconda3\envs\mpytc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Load Data

In [4]:
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')
submission_df = pd.read_csv('dataset/sample_submission.csv')

train_df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


## 3. Prepare Text and Generate Embeddings

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def prepare_text(df):
    # Clean up the prompt column
    df['prompt'] = df['prompt'].apply(lambda x: re.sub(r'[""|""]', '', str(x)))
    df['text_a'] = df['prompt'] + ' ' + df['response_a']
    df['text_b'] = df['prompt'] + ' ' + df['response_b']
    return df

train_df = prepare_text(train_df)
test_df = prepare_text(test_df)

# Generate embeddings
train_embeddings_a = model.encode(train_df['text_a'].tolist(), show_progress_bar=True)
train_embeddings_b = model.encode(train_df['text_b'].tolist(), show_progress_bar=True)
test_embeddings_a = model.encode(test_df['text_a'].tolist(), show_progress_bar=True)
test_embeddings_b = model.encode(test_df['text_b'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:00<00:00, 125.04it/s]


## 4. Feature Engineering

In [6]:
# Use the difference between embeddings as features
X_train = train_embeddings_a - train_embeddings_b
X_test = test_embeddings_a - test_embeddings_b

## 5. Model Training

In [7]:
# Create target variable
conditions = [
    train_df['winner_model_a'] == 1,
    train_df['winner_model_b'] == 1,
    train_df['winner_tie'] == 1
]
choices = [0, 1, 2] # 0: model_a, 1: model_b, 2: tie
y_train = np.select(conditions, choices, default=-1)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


## 6. Prediction

In [8]:
predictions = classifier.predict_proba(X_test)

## 7. Submission

In [9]:
submission_df['winner_model_a'] = predictions[:, 0]
submission_df['winner_model_b'] = predictions[:, 1]
submission_df['winner_tie'] = predictions[:, 2]

submission_df.to_csv('submission_embedding.csv', index=False)

submission_df.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.439387,0.240070,0.320544
1,211333,0.249852,0.453601,0.296547
2,1233961,0.295781,0.381972,0.322247
